In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

from py.is_rebalance import is_rebalance
from py.signals import signals
from py.split_df import split_df
from py.simulate import simulate

In [2]:
# VARIABLES

# We want 180 day windows (6 months) with 120 day overlap (4 months)
# Since our dataframe is in hours, multiply by 24
window_len = 24 * 180
overlap = 24 * 120

# Assets traded
assets = ['ETH', 'USD']

# Moving average intervals used
moving_averages = [50, 100, 200]

# Potential ETH to DAI allocations from bullish signals
bull_allocation = [
    [0.90, 0.10],
    [0.85, 0.15],
    [0.80, 0.20],
    [0.75, 0.25],
    [0.70, 0.30],
    [0.65, 0.35],
    [0.60, 0.40]
]

# List of allocations used, with the inverse allocation for bearish signals
allocation_lst = [{'bull': b,
                   'neutral': [0.50, 0.50],
                   'bear': b[::-1]}
                  for b in bull_allocation]


# Minimum difference in weighting needed to rebalance without a new signal
# This prevents unnecessary rebalancing
wiggle_room_lst = np.arange(0, 0.11, 0.01)

In [3]:
df = pd.read_csv('../data/ETH-USDT.csv', usecols=['date', 'close']).rename({'close':'ETH'}, axis=1)
df['date'] = pd.to_datetime(df['date'])
df['USD'] = 1

# Create columns to 
df['rebalance'] = is_rebalance(df['date'])
df['signal'] = signals(df['ETH'], df['rebalance'], *moving_averages)

# Split dataframe into windows 
dfs = split_df(df.to_dict(orient='records'), overlap, window_len)

results = []

for allocation in allocation_lst:
    for wiggle_room in wiggle_room_lst:
        result = {
            'wiggle_room': wiggle_room,
            'allocation': '/'.join(str(x) for x in allocation['bull']),
        }

        # Add result for each split dataframe
        for df_split in dfs:
            start = datetime.strftime(df_split[0]['date'], '%Y.%m.%d')
            end = datetime.strftime(df_split[-1]['date'], '%Y.%m.%d')

            _, _, performance = simulate(assets, allocation, wiggle_room, df_split)

            result[start + '-' + end] = performance

        # Save result to results
        results.append(result)


# Convert dict to dataframe
df_results = pd.DataFrame.from_records(results)
df_results['sum'] = df_results.drop(['wiggle_room', 'allocation'], axis=1).sum(axis=1)

# Sort
df_results = df_results.sort_values('sum', ascending=False)


# Save signals and performance to CSV
df.to_csv('backtests/signals.csv', index=False)
df_results.to_csv('backtests/performance.csv', index=False)

In [4]:
df_results

,wiggle_room,allocation,2017.08.16-2018.02.14,2017.10.16-2018.04.15,2017.12.15-2018.06.14,2018.02.14-2018.08.14,2018.04.15-2018.10.13,2018.06.14-2018.12.12,2018.08.14-2019.02.10,2018.10.13-2019.04.11,2018.12.12-2019.06.11,2019.02.10-2019.08.10,2019.04.11-2019.10.09,2019.06.11-2019.12.08,2019.08.09-2020.02.06,2019.10.09-2020.04.06,sum
10,0.10,0.9/0.1,-747.325679,-332.360418,752.221696,2073.866756,1221.497841,2455.230904,1919.909730,1673.398288,-1089.125145,-825.977314,-34.806333,793.930676,548.706342,563.204556,8972.371900
9,0.09,0.9/0.1,-803.673218,-332.360418,752.221696,2054.723307,1195.138312,2393.513259,1758.102824,1618.018982,-1089.125145,-825.977314,-34.806333,793.930676,546.720891,530.020698,8556.448217
8,0.08,0.9/0.1,-803.673218,-332.360418,752.221696,2054.723307,1195.138312,2393.513259,1758.102824,1618.018982,-1098.977227,-828.282679,-44.027735,765.308983,540.511454,499.349290,8469.566830
7,0.07,0.9/0.1,-803.673218,-360.250382,678.418562,2081.527164,1212.786722,2378.054235,1730.436916,1603.664100,-1115.961217,-828.282679,-44.027735,765.308983,540.511454,499.349290,8337.862194
6,0.06,0.9/0.1,-790.201782,-333.339089,737.853552,2059.360342,1162.105398,2303.586440,1775.176552,1603.274579,-1116.107380,-831.527922,-56.203349,765.308983,540.511454,508.973436,8328.771215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.09,0.6/0.4,-629.543150,-549.460872,180.962084,1544.295992,871.978215,2030.625430,1553.994221,1123.080768,-853.922984,-840.126293,-354.277402,527.937261,583.181439,314.626227,5503.350937
73,0.07,0.6/0.4,-627.484295,-682.495552,184.874410,1562.691810,885.166743,1993.110404,1597.759780,1123.080768,-873.665249,-832.515296,-339.678297,582.520557,583.181439,313.657948,5470.205172
76,0.10,0.6/0.4,-629.543150,-549.549555,76.330106,1544.295992,870.867200,2030.625430,1553.994221,1123.080768,-849.259419,-760.173945,-353.895492,527.937261,608.642908,259.938876,5453.291202
72,0.06,0.6/0.4,-627.484295,-598.417830,184.874410,1562.691810,885.917162,1993.110404,1390.457514,1119.381743,-857.697623,-860.114498,-339.951357,588.430262,583.448552,283.494258,5308.140514


### Conclusion: Fart Set will adjust ETH allocation to 90%-10% for long/short signals.  Additional rebalances with the same allocation require a 10% weight difference to trigger.